In [27]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("suraj520/customer-support-ticket-dataset")

# print("Path to dataset files:", path)

In [41]:
# pip install openai-agents
import os
import pandas as pd

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser



In [45]:
# sup_ticks = pd.read_csv("customer_support_tickets.csv")

# Twitter support dataset from kaggle https://www.kaggle.com/datasets/thoughtvector/customer-support-on-twitter?resource=download
### LIMIT the tweets to new inbound support requests
twit_sup = pd.read_csv("twitter_support.csv")
new_inbound = twit_sup[(pd.isna(twit_sup['in_response_to_tweet_id']) & twit_sup['inbound']==True)].reset_index(drop=True)
new_inbound.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,119237,105834,True,Wed Oct 11 06:55:44 +0000 2017,@AppleSupport causing the reply to be disregar...,119236,NaN
1,119239,105835,True,Wed Oct 11 13:00:09 +0000 2017,@76328 I really hope you all change but I'm su...,119238,NaN
2,119250,105838,True,Wed Oct 11 05:33:17 +0000 2017,"@AppleSupport hi #apple, I’ve a concern about ...","119249,119251",NaN
3,119253,105839,True,Wed Oct 11 07:21:34 +0000 2017,I just updated my phone and suddenly everythin...,119252,NaN
4,119256,105840,True,Wed Oct 11 12:53:29 +0000 2017,@76495 @91226 Please help! Spotify Premium ski...,119254,NaN


In [ ]:

## Set keys in jupyter environment 
# https://stackoverflow.com/questions/70372120/how-to-use-api-key-without-directly-using-it-in-the-python-code
%env OPENAI_API_KEY <your_open_ai_key>
# %env TAVILY_API_KEY

print(os.getenv('OPENAI_API_KEY'))
# print(os.getenv('TAVILY_API_KEY'))

In [29]:
llm = ChatOpenAI()

In [32]:
# Define the prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a customer support bot equipped to respond to issues for a wide variety of brands. The relevant brand could be noted with an @ sign."),
    ("user", "{input}")
])

In [62]:
# Chain together prompt and llm
chain = prompt | llm 
chain.invoke({"input": new_inbound['text'][2]})

AIMessage(content="I'm sorry to hear that you're experiencing performance issues with your iPhone 6 after updating to the latest iOS. Here are a few steps you can take to try and improve the performance:\n\n1. **Restart your iPhone**: Sometimes a simple restart can help improve performance.\n\n2. **Check for Software Updates**: Make sure you have the latest version of iOS installed on your device as newer updates may contain performance improvements.\n\n3. **Reset Settings**: You can try resetting all settings on your iPhone by going to Settings > General > Reset > Reset All Settings. This won't erase your data but will reset all settings to default.\n\n4. **Clear Storage**: If your device is low on storage, try clearing out unnecessary data such as apps, photos, or videos to free up space.\n\n5. **Contact Apple Support**: If the issue persists, I recommend reaching out to @AppleSupport directly for further assistance. They may have additional troubleshooting steps or solutions tailore

In [61]:
# Use output parser to get just the bots response
output_parser = StrOutputParser()
chain = prompt | llm | output_parser
chain.invoke({"input": new_inbound['text'][2]})

'I understand your concern about the performance of iOS on your iPhone 6. Here are a few steps you can try to improve the speed:\n\n1. Make sure your device is updated to the latest version of iOS.\n2. Clear up storage space by deleting unused apps, photos, and videos.\n3. Restart your iPhone by turning it off and then on again.\n4. Reset all settings by going to Settings > General > Reset > Reset All Settings.\n5. If the issue persists, you may consider contacting Apple Support for further assistance.\n\nI hope these suggestions help improve the performance of your iPhone 6. Let me know if you need further assistance!'

In [54]:
# go through all of the inbound support requests and add an AI response
new_inbound['AI_Response'] = new_inbound['text'].apply(lambda x: chain.invoke({"input":x}))


In [60]:
# Loop through the dataframe and print the support requests and responses
for index, row in new_inbound.iterrows():
    print(f"Ticket Number: {index}\n Support Request: {row['text']}\n\n, AI Response: {row['AI_Response']}")
    print("------ \n")

Ticket Number: 0
 Support Request: @AppleSupport causing the reply to be disregarded and the tapped notification under the keyboard is opened😡😡😡

, AI Response: I'm sorry to hear about the issue you're experiencing with the @AppleSupport app. It sounds like you're having trouble with notifications opening when you tap on them. Have you tried restarting the app or your device to see if that helps resolve the issue? If the problem persists, you may want to reach out to Apple Support directly for further assistance.
------ 

Ticket Number: 1
 Support Request: @76328 I really hope you all change but I'm sure you won't! Because you don't have to!

, AI Response: I'm here to help you with any issues you might be experiencing with a particular brand. Please let me know what specific problem you are facing so that I can assist you accordingly.
------ 

Ticket Number: 2
 Support Request: @AppleSupport hi #apple, I’ve a concern about the latest ios is too slow on #iphone6 and i am not happy with